In [1]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import time
from datetime import datetime, timedelta
from IPython.display import clear_output, display

def get_eurcad_data(interval="5m", period="1d"):
    try:
        eurcad = yf.Ticker("EURCAD=X")
        data = eurcad.history(period=period, interval=interval)
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def calculate_sma(data, window):
    return data['Close'].rolling(window=window).mean()

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.rolling(window=window).mean()
    ema_down = down.rolling(window=window).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

def calculate_macd(data):
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20):
    sma = calculate_sma(data, window)
    std = data['Close'].rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, lower_band

def find_support_resistance(data):
    support = data['Low'].rolling(window=10).min()
    resistance = data['High'].rolling(window=10).max()
    return support, resistance

def analyze_and_generate_signals(data):
    if data is None or data.empty or len(data) < 20:
        return None, None
    
    sma_short = calculate_sma(data, window=9)
    sma_long = calculate_sma(data, window=18)
    rsi = calculate_rsi(data, window=14)
    macd, signal = calculate_macd(data)
    upper_band, lower_band = calculate_bollinger_bands(data)
    support, resistance = find_support_resistance(data)
    
    last_price = data['Close'].iloc[-1]
    current_time = datetime.utcnow() - timedelta(hours=3)  # UTC -3
    trade_time = (current_time + timedelta(minutes=5)).strftime("%H:%M")
    
    # Determine signal type
    signal_type = None
    if sma_short.iloc[-1] > sma_long.iloc[-1] and rsi.iloc[-1] < 70:
        signal_type = "CALL 🟩"
    elif sma_short.iloc[-1] < sma_long.iloc[-1] and rsi.iloc[-1] > 30:
        signal_type = "PUT 🟥"
    
    if macd.iloc[-1] > signal.iloc[-1] and macd.iloc[-2] <= signal.iloc[-2]:
        signal_type = "CALL 🟩"
    elif macd.iloc[-1] < signal.iloc[-1] and macd.iloc[-2] >= signal.iloc[-2]:
        signal_type = "PUT 🟥"
    
    if last_price < lower_band.iloc[-1]:
        signal_type = "CALL 🟩"
    elif last_price > upper_band.iloc[-1]:
        signal_type = "PUT 🟥"
    
    if last_price <= support.iloc[-1]:
        signal_type = "CALL 🟩"
    elif last_price >= resistance.iloc[-1]:
        signal_type = "PUT 🟥"
    
    if signal_type:
        signal_text = f"💰 5 minutes expiry\nEUR/CAD;{trade_time};{signal_type}\n🕐 TIME TO {trade_time}"  
        return signal_text, trade_time
    return None, None

def create_candlestick_chart(data, title):
    if data is None or data.empty:
        print(f"No data to display for {title}.")
        return None
    
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False)
    return fig

def update_charts_and_signals():
    signals_history = []
    try:
        while True:
            data_5min = get_eurcad_data(interval="5m")
            fig_5min = create_candlestick_chart(data_5min, "EUR/CAD 5-Minute Chart")
            
            if fig_5min is not None:
                signal_text, trade_time = analyze_and_generate_signals(data_5min)
                
                if signal_text:
                    signals_history.append(signal_text)
                    signals_history = signals_history[-5:]
                    clear_output(wait=True)
                    display(fig_5min)
                    print("Trading Signals:")
                    for signal in signals_history:
                        print(signal)
                else:
                    clear_output(wait=True)
                    display(fig_5min)
                    print("No New Trading Signals")
            
            time.sleep(600)  # 10-minute interval
    except KeyboardInterrupt:
        print("Chart update stopped.")

if __name__ == "__main__":
    update_charts_and_signals()


KeyboardInterrupt: 

In [5]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import time
from datetime import datetime, timedelta
from IPython.display import clear_output, display

def get_eurcad_data(interval="5m", period="1d"):
    try:
        eurcad = yf.Ticker("EURCAD=X")
        data = eurcad.history(period=period, interval=interval)
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def calculate_sma(data, window):
    return data['Close'].rolling(window=window).mean()

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.rolling(window=window).mean()
    ema_down = down.rolling(window=window).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

def calculate_macd(data):
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20):
    sma = calculate_sma(data, window)
    std = data['Close'].rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, lower_band

def find_support_resistance(data):
    support = data['Low'].rolling(window=10).min()
    resistance = data['High'].rolling(window=10).max()
    return support, resistance

def calculate_pivot_points(data):
    pivot = (data['High'] + data['Low'] + data['Close']) / 3
    r1 = (pivot * 2) - data['Low']
    s1 = (pivot * 2) - data['High']
    return pivot, r1, s1

def analyze_and_generate_signals(data):
    if data is None or data.empty or len(data) < 20:
        return None, None
    
    sma_short = calculate_sma(data, window=9)
    sma_long = calculate_sma(data, window=18)
    rsi = calculate_rsi(data, window=14)
    macd, signal = calculate_macd(data)
    upper_band, lower_band = calculate_bollinger_bands(data)
    support, resistance = find_support_resistance(data)
    pivot, r1, s1 = calculate_pivot_points(data)
    
    last_price = data['Close'].iloc[-1]
    current_time = datetime.utcnow() - timedelta(hours=3)  # UTC -3
    trade_time = (current_time + timedelta(minutes=5)).strftime("%H:%M")
    
    signal_type = None
    if sma_short.iloc[-1] > sma_long.iloc[-1] and rsi.iloc[-1] < 70:
        signal_type = "CALL 🟩"
    elif sma_short.iloc[-1] < sma_long.iloc[-1] and rsi.iloc[-1] > 30:
        signal_type = "PUT 🟥"
    
    if macd.iloc[-1] > signal.iloc[-1] and macd.iloc[-2] <= signal.iloc[-2]:
        signal_type = "CALL 🟩"
    elif macd.iloc[-1] < signal.iloc[-1] and macd.iloc[-2] >= signal.iloc[-2]:
        signal_type = "PUT 🟥"
    
    if last_price < lower_band.iloc[-1]:
        signal_type = "CALL 🟩"
    elif last_price > upper_band.iloc[-1]:
        signal_type = "PUT 🟥"
    
    if last_price <= support.iloc[-1]:
        signal_type = "CALL 🟩"
    elif last_price >= resistance.iloc[-1]:
        signal_type = "PUT 🟥"
    
    if last_price < s1.iloc[-1]:
        signal_type = "CALL 🟩"
    elif last_price > r1.iloc[-1]:
        signal_type = "PUT 🟥"
    
    if signal_type:
        signal_text = f"💰 5 minutes expiry\nEUR/CAD;{trade_time};{signal_type}\n🕐 TIME TO {trade_time}"  
        return signal_text, trade_time
    return None, None

def create_candlestick_chart(data, title):
    if data is None or data.empty:
        print(f"No data to display for {title}.")
        return None
    
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False)
    return fig

def update_charts_and_signals():
    signals_history = []
    try:
        while True:
            data_5min = get_eurcad_data(interval="5m")
            fig_5min = create_candlestick_chart(data_5min, "EUR/CAD 5-Minute Chart")
            
            if fig_5min is not None:
                signal_text, trade_time = analyze_and_generate_signals(data_5min)
                
                if signal_text:
                    signals_history.append(signal_text)
                    signals_history = signals_history[-5:]
                    clear_output(wait=True)
                    display(fig_5min)
                    print("Trading Signals:")
                    for signal in signals_history:
                        print(signal)
                else:
                    clear_output(wait=True)
                    display(fig_5min)
                    print("No New Trading Signals")
            
            time.sleep(600)  # 10-minute interval
    except KeyboardInterrupt:
        print("Chart update stopped.")

if __name__ == "__main__":
    update_charts_and_signals()


Trading Signals:
💰 5 minutes expiry
EUR/CAD;05:09;CALL 🟩
🕐 TIME TO 05:09
💰 5 minutes expiry
EUR/CAD;05:19;PUT 🟥
🕐 TIME TO 05:19
💰 5 minutes expiry
EUR/CAD;05:29;PUT 🟥
🕐 TIME TO 05:29
💰 5 minutes expiry
EUR/CAD;05:39;CALL 🟩
🕐 TIME TO 05:39
💰 5 minutes expiry
EUR/CAD;05:49;PUT 🟥
🕐 TIME TO 05:49
